In [25]:
import selenium
import pandas as pd
import numpy as np
from datetime import datetime

# Now that the subtitles are all already available on my folder, let's select every single .srt file that we've been able to extract from the zip files downloaded using the wget method on the command line.

In [274]:
a = !ls *srt

In [275]:
a[:10]

['(1971) Escape from the Planet of the Apes 1080p H.264 Multi (moviesbyrizzo).srt',
 '(1973) Battle for the Planet of the Apes 1080p H.264 Multi (moviesbyrizzo).srt',
 '(Blue) Gifted.Hands.The.Ben.Carson.Story.2009.720p.WEB-HD.x264.eng.srt',
 '(Green) Gifted.Hands.The.Ben.Carson.Story.2009.720p.WEB-HD.x264.eng.srt',
 '(Purple) Gifted.Hands.The.Ben.Carson.Story.2009.720p.WEB-HD.x264.eng.srt',
 '(Red) Gifted.Hands.The.Ben.Carson.Story.2009.720p.WEB-HD.x264.eng.srt',
 '(Yellow) Gifted.Hands.The.Ben.Carson.Story.2009.720p.WEB-HD.x264.eng.srt',
 '00003.srt',
 '00003Eng.srt',
 '007.From.Russia.With.Love.1963.HDrip.x264-450MB.srt']

In [78]:
df = pd.read_csv('mymoviedb.csv',error_bad_lines=False, engine="python")

<ipython-input-78-a3d5f548d8cb>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('mymoviedb.csv',error_bad_lines=False, engine="python")


In [235]:
df.head()

,Release_Date,Title,Overview,Popularity,Vote_Count,Vote_Average,Original_Language,Genre,Poster_Url
0,2021-12-15,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,5083.954,8940,8.3,en,"Action, Adventure, Science Fiction",https://image.tmdb.org/t/p/original/1g0dhYtq4i...
1,2022-03-01,The Batman,"In his second year of fighting crime, Batman u...",3827.658,1151,8.1,en,"Crime, Mystery, Thriller",https://image.tmdb.org/t/p/original/74xTEgt7R3...
2,2022-02-25,No Exit,Stranded at a rest stop in the mountains durin...,2618.087,122,6.3,en,Thriller,https://image.tmdb.org/t/p/original/vDHsLnOWKl...
3,2021-11-24,Encanto,"The tale of an extraordinary family, the Madri...",2402.201,5076,7.7,en,"Animation, Comedy, Family, Fantasy",https://image.tmdb.org/t/p/original/4j0PNHkMr5...
4,2021-12-22,The King's Man,As a collection of history's worst tyrants and...,1895.511,1793,7.0,en,"Action, Adventure, Thriller, War",https://image.tmdb.org/t/p/original/aq4Pwv5Xeu...


In [273]:
df[df['Title'] == 'The United States vs. Billie Holiday']

,Release_Date,Title,Overview,Popularity,Vote_Count,Vote_Average,Original_Language,Genre,Poster_Url
9835,2021-03-31,The United States vs. Billie Holiday,Billie Holiday spent much of her career being ...,13.354,152,6.7,en,"Music, Drama, History",https://image.tmdb.org/t/p/original/vEzkxuE2sJ...


## The only possible way to merge 2 Dataframes is if at least one of the columns on both of them are equal. For that, we'll need to find a way to figure out which subtitle file corresponds to each row of the Dataset. I've decided to do that by creating a list of words both for the column "Titles" and "Release_Date" of the dataframe, as well as for the .srt file names.

In [276]:
parted_names = {}
agora = datetime.now()
for i in a:
    nova_string = ''
    for position,charac in enumerate(i):
        if charac.isupper():
            nova_string += ' '
        nova_string += charac

    parted_names[i] = str.split(
                        str.lower(
                                    nova_string.replace('.',' ')\
                                    .replace('_', ' ')\
                                    .replace('[',' ')\
                                    .replace(']', ' ')\
                                    .replace('@','')\
                                    .replace('-',' ')\
                                    .replace('(',' ')\
                                    .replace(')',' ')
                                )
                    )
    
print(f'Time it took = {datetime.now() - agora}')

Time it took = 0:00:00.105674


## An example of how we're partitioning them

In [277]:
list(parted_names.values())[:3]

[['1971',
  'escape',
  'from',
  'the',
  'planet',
  'of',
  'the',
  'apes',
  '1080p',
  'h',
  '264',
  'multi',
  'moviesbyrizzo',
  'srt'],
 ['1973',
  'battle',
  'for',
  'the',
  'planet',
  'of',
  'the',
  'apes',
  '1080p',
  'h',
  '264',
  'multi',
  'moviesbyrizzo',
  'srt'],
 ['blue',
  'gifted',
  'hands',
  'the',
  'ben',
  'carson',
  'story',
  '2009',
  '720p',
  'w',
  'e',
  'b',
  'h',
  'd',
  'x264',
  'eng',
  'srt']]

## Now I'm gonna show you how I'm partitioning the Title and Release_Date columns of the Kaggle Dataframe

In [233]:
movie_name =str.lower(df['Title'][6] + ' ' + df['Release_Date'][6][:4]).replace(':','').replace('-',' ')
movie_name.split()

['scream', '2022']

# Now we'll see if every single name on the list up until when the release date appears is the same on both lists created (actually, a dict and a list...)
## Now I'm finally figuring out the corresponding file name to every single .srt file that's been downloaded!

In [279]:
from datetime import datetime
correspondencias = {}
agora = datetime.now()
for keys, values in parted_names.items():
    print('-'*20 + keys + '-'*20)
    for i,v in enumerate(df['Title'].tolist()):
        try:
            movie_name =str.lower(v + ' ' + df['Release_Date'][i][:4]).replace(':','').replace('-',' ').replace("'","").split()
            ano_index = -1
            for i,vs in enumerate(movie_name):
                try:
                    if int(vs) >1950 and int(vs) < 2023:
                         ano_index = i
                except:
                    pass
            if movie_name[:ano_index] == values[:ano_index]:
                correspondencias[v] = keys
        except TypeError:
            pass
print(datetime.now() - agora)

--------------------(1971) Escape from the Planet of the Apes 1080p H.264 Multi (moviesbyrizzo).srt--------------------
--------------------(1973) Battle for the Planet of the Apes 1080p H.264 Multi (moviesbyrizzo).srt--------------------
--------------------(Blue) Gifted.Hands.The.Ben.Carson.Story.2009.720p.WEB-HD.x264.eng.srt--------------------
--------------------(Green) Gifted.Hands.The.Ben.Carson.Story.2009.720p.WEB-HD.x264.eng.srt--------------------
--------------------(Purple) Gifted.Hands.The.Ben.Carson.Story.2009.720p.WEB-HD.x264.eng.srt--------------------
--------------------(Red) Gifted.Hands.The.Ben.Carson.Story.2009.720p.WEB-HD.x264.eng.srt--------------------
--------------------(Yellow) Gifted.Hands.The.Ben.Carson.Story.2009.720p.WEB-HD.x264.eng.srt--------------------
--------------------00003.srt--------------------
--------------------00003Eng.srt--------------------
--------------------007.From.Russia.With.Love.1963.HDrip.x264-450MB.srt--------------------
-------

--------------------????.Animal.World.2018.1080p.NF.WEB-DL.DDP5.1.x264-Ao.eng.srt--------------------
--------------------????.Armour.of.God.1986.720p.BluRay.x264.AAC TYZH.????.srt--------------------
--------------------?????.Zootopia.2016.HD1080P.X264.AAC.English&Mandarin.CHS.Mp4Ba.srt--------------------
--------------------??????? ??????? - The Red Ghost 2020 1080p Blu-ray REMUX AVC DTS-HD MA 5.1-AYA.srt--------------------
--------------------???????? ???? ?????????  Lady Chatterley's lover (2015).eng.srt--------------------
--------------------??N?.A.Fantastic.Woman.2017.HD1080P.X264.AAC.Spanish.CHS..srt--------------------
--------------------@PGP_Venom_Let_There_Be_Carnage_2021_HDTS_c1nem4_SUNSCREEN_GM (2).srt--------------------
--------------------A Cinderella Story If the Shoe Fits 2016 1080p HMAX WEBRip DD5 1 x264-MELON.srt--------------------
--------------------A Cinderella Story Once Upon a Song (2011).en.srt--------------------
--------------------A Common Man 2012 1080

--------------------About.Cherry.2012.720p.BluRay.x264.[MixMovies].srt--------------------
--------------------About.Time.2013.720p.BluRay.x264.YIFY.srt--------------------
--------------------Above The Rim.srt--------------------
--------------------Above.the.Shadows.2019.1080p.WEB-DL.H264.AC3-EVO.srt--------------------
--------------------Abraham_Lincoln_Vs_Zombies_2012_BRRip_XViD_AC3-26k.srt--------------------
--------------------Absolutely.Anything.2015.BRRip.XviD.AC3-EVO.srt--------------------
--------------------AbsolutelyFabulousTheMovie.brrip.2016.720p.srt--------------------
--------------------Accidental Love (2015).srt--------------------
--------------------According to Greta (2009) BluRay.720p.x264-THUGLiNE.srt--------------------
--------------------Ace.Ventura.When.Nature.Calls.1995.720p.BluRay.X264-AMIABLE.srt--------------------
--------------------Act.of.Valor.2012.english.srt--------------------
--------------------Ad.Astra.2019.720p.HDCAM-GETB8-HI.srt------------

--------------------American.Ninja.4.The.Annihilation.1990.720p.BluRay.x264-[YTS.AG].srt--------------------
--------------------American.Psycho.II.All.American.Girl.2002.1080p.BluRay.x264-BRMP.srt--------------------
--------------------American.Satan.2018.1080p.WEB-DL.H264.AC3-EVO.srt--------------------
--------------------American.Ultra.2015.720p.BRRip.x264.AAC-ETRG.srt--------------------
--------------------AmericanHustle.2013.BRRip.srt--------------------
--------------------AmericanPastoral.brrip.2016.720p.srt--------------------
--------------------AmericanPiePresentsTheBookOfLove.brrip.2009.1080p.srt--------------------
--------------------Amityville.II.The.Possession.1982.720p.BluRay.x264-PSYCHD.srt--------------------
--------------------Amityville.The.Awakening.2017.720p.BluRay.x264-YTS.AG.srt--------------------
--------------------Ammonite.2020.720p.AMZN.WEB-DL.sdh.srt--------------------
--------------------Amores perros (Mexico 2000).srt--------------------
-----------

--------------------Astro.Boy.2009.1080p.BluRay.H264.AAC-RARBG.srt--------------------
--------------------AstВrix Chez Les Bretons (1986).srt--------------------
--------------------AstВrix Et ClВopГtre (1968).srt--------------------
--------------------At.the.End.of.the.Tunnel.2016.720p.BluRay.x264-BiPOLAR.srt--------------------
--------------------AtomicBlonde.brrip.2017.1080p.srt--------------------
--------------------Atonement.2007.iNTERNAL.BDRip.x264-MARS.srt--------------------
--------------------Attack.on.Titan.Part.2.End.of.the.World.2015.1080p.WEB-DL.x264-NoGRP.srt--------------------
--------------------Attack.the.Block.2011.720p.BrRip.x264.YIFY.srt--------------------
--------------------Au.revoir.les.enfants.1987.BluRay.720p.AC3.x264-CHD.eng.srt--------------------
--------------------Audible.2021.1080p.WEB.H264-BIGDOC-EN-FORCED.srt--------------------
--------------------Audible.2021.1080p.WEB.H264-BIGDOC.srt--------------------
--------------------August.Osage.County.

...

--------------------Witness.for.the.Prosecution.1957.720p.BluRay.X264-AMIABLE.srt--------------------
--------------------Wizards.of.Waverly.Place.The.Movie.2009.1080p.AMZN.WEB-DL.DD+5.1.x264-QOQ.eng.srt--------------------
--------------------Wolfwalkers.1080p.ATVP.WEB-DL.DDP5.1.Atmos.x264-CM-HI.srt--------------------
--------------------Woman.Walks.Ahead.2018.HDRip.AC3.X264-CMRG-HI.srt--------------------
--------------------Wonder.Boys.2000.720p.HDTV.x264.DD2.0-FGT.srt--------------------
--------------------Wonder.Park.2019.WEB-DL.x264-FGT.srt--------------------
--------------------Wonder.Woman.Bloodlines.2019.HDRip.XviD.AC3-EVO.srt--------------------
--------------------Woodlawn.2015.BRRip.XviD.AC3-EVO.srt--------------------
--------------------Woody.Woodpecker.2017.1080p.BluRay.x264-CADAVER.srt--------------------
--------------------Words.on.Bathroom.Walls.2020.1080p.AMZN.WEB-DL.DDP5.1.H.264-EVO-HI.srt--------------------
--------------------Work.It.2020.WEBRip-HI.srt-------

--------------------citizen-kane-yify-english.srt--------------------
--------------------city of women 720p.srt--------------------
--------------------cold-mountain-yify-english.srt--------------------
--------------------college.road.trip.eng.srt--------------------
--------------------colombiana-yify-english.srt--------------------
--------------------confessions.of.a.teenage.drama.queen.2004.1080p.hdtv.x264-regret.eng.srt--------------------
--------------------cosifantutte72ray.Ganool.com.srt--------------------
--------------------cr2tfj-extd-1080p-illsn.srt--------------------
--------------------creep-htcommupcsd.srt--------------------
--------------------crocodile-dundee-ii-yify-english.srt--------------------
--------------------daa-broken.city-1080p.eng-hi.srt--------------------
--------------------dances-with-wolves-yify-english.srt--------------------
--------------------dantes-peak-yify-english.srt--------------------
--------------------dawn-of-the-planet-of-the-apes.

--------------------kiss.and.cry.2017.720p.webrip.x264-strife.srt--------------------
--------------------korea The Beast 2019 720p.srt--------------------
--------------------la-confidential-yify-english.srt--------------------
--------------------lady bird (2017).srt--------------------
--------------------laggies.720p.BluRay.x264.YIFY.srt--------------------
--------------------lay-the-favorite-yify-english.srt--------------------
--------------------le.chef.2012.bdrip.x264-nodlabs.srt--------------------
--------------------leprechaun-origins.720p.BluRay.x264.YIFY.srt--------------------
--------------------licence-to-kill-yify-english.srt--------------------
--------------------lilo-stitch-yify-english.srt--------------------
--------------------little-fockers-yify-english.srt--------------------
--------------------little.buddha.1993.720p.bluray.x264-usury.srt--------------------
--------------------loerdofwar72ray.Ganool.com.srt--------------------
--------------------loveguru-d

--------------------the-6th-day-yify-english.srt--------------------
--------------------the-anomaly.720p.BluRay.x264.YIFY.srt--------------------
--------------------the-atticus-institute.720p.BluRay.x264.YIFY.srt--------------------
--------------------the-babadook.720p.BluRay.x264.YIFY.srt--------------------
--------------------the-blues-brothers-yify-english.srt--------------------
--------------------the-bodyguard-yify-english.srt--------------------
--------------------the-bone-collector-yify-english.srt--------------------
--------------------the-bourne-identity-yify-english.srt--------------------
--------------------the-burbs-yify-english.srt--------------------
--------------------the-constant-gardener-yify-english.srt--------------------
--------------------the-counselor-yify-english.srt--------------------
--------------------the-d-train.720p.BluRay.x264.YIFY.srt--------------------
--------------------the-day-after-tomorrow-yify-english.srt--------------------
-----------

--------------------zathura-a-space-adventure-yify-english.srt--------------------
--------------------zombeavers.720p.BluRay.x264.YIFY.srt--------------------
--------------------zombie.shark.2015.720p.hdtv.srt--------------------
--------------------┐╦└│─м╖Є╕╛.Kramer.Vs.Kramer.1979.BDRip.2Audio.MiniSD-TLF_track6_eng.srt--------------------
0:08:12.330181


## Here's the final result hehe

In [281]:
correspondencias

{'RED': 'red-riding-hood-yify-english.srt',
 '102 Dalmatians': '102.Dalmatians.2000.1080p.WEBRip.DD5.1.x264-NTb.srt',
 '10x10': '10x10.2018.WEBRip.x264-FGT.srt',
 '11-11-11': '11.11.11.2011.720p.BluRay.H264.AAC-RARBG Hi Eng.srt',
 '12 Strong': '12 Strong 2018 BDRip x264-GECKOS.srt',
 '12 Angry Men': '12.Angry.Men.1957.DVDRip.x264-DJ.srt',
 '12 Hour Shift': '12.Hour.Shift.2020.HDRip.XviD.AC3-EVO-HI.srt',
 '12 Mighty Orphans': '12.Mighty.Orphans.2021.HDCAM.850MB.x264-SUNSCREEN-HI.srt',
 '12 Years a Slave': '12.YearsASlave.2013.BRRip.srt',
 '127 Hours': '127.Hours.2010.MULTi.WiTH.TRUEFRENCH.1080p.BluRay.x264.DTS-LOST.srt',
 '13 Ghosts': '13.Ghosts.1960.iNTERNAL.BDRip.x264-LiBRARiANS.srt',
 '13': '13_Spanish.srt',
 '13 Sins': '13.Sins.2014.1080p.BluRay.x264.YIFY.srt',
 '14 Peaks: Nothing Is Impossible': '14.Peaks.Nothing.Is.Impossible.2021.720p.WEBRip.x264.AAC-[YTS.MX].srt',
 '1408': '1408[2007]DvDrip[Eng]-aXXo.srt',
 '1492: Conquest of Paradise': '1492.Conquest.of.Paradise.1992.BRRip.XviD

# If you wanna understand the block of code above, I think you should start by understanding this one. You don't need to though, it's just an extra...

In [212]:
movie6 = str.lower(df['Title'].tolist()[6] + ' ' + df['Release_Date'].tolist()[6][:4]).split()
ano_index = -1
for i,v in enumerate(movie6):
    try:
        if int(v) >1950 and int(v) < 2023:
             ano_index = i
    except:
        pass
if ano_index = -1:
    
    
correspondencia1 = {}
if movie6[:ano_index] == l['Scream.2022.HDCAM.850MB.c1nem4.x264-SUNSCREEN.srt'][:ano_index]:
    correspondencia1[]

True

In [282]:
new_df_dict = {}
n_arquivos_baixados = 0
for i,v in correspondencias.items():
    try:
        new_df_dict[i] = [open(v,'r').read()]
        n_arquivos_baixados +=1
    except:
        pass
print(f'Arquivos baixados: {n_arquivos_baixados}')

Arquivos baixados: 3355


In [283]:
df_subs = pd.DataFrame(new_df_dict)\
            .transpose()\
            .rename(columns = {0:'Subs'})\
            .reset_index()\
            .rename(columns = {'index':'Title'})

In [284]:
pd.merge(df_subs,df)

,Title,Subs,Release_Date,Overview,Popularity,Vote_Count,Vote_Average,Original_Language,Genre,Poster_Url
0,102 Dalmatians,"0\n00:00:00,150 --> 00:00:4,666\n<b><font face...",2000-10-07,Get ready for a howling good time as an all ne...,14.874,1106,5.5,en,"Family, Comedy",https://image.tmdb.org/t/p/original/dSxnIika9y...
1,10x10,"﻿1\n00:01:45,844 --> 00:01:48,412\nHey, guys, ...",2018-04-13,"Lewis is an outwardly ordinary guy, but in rea...",21.336,441,5.3,en,Thriller,https://image.tmdb.org/t/p/original/egMETBYual...
2,11-11-11,"1\n00:00:06,220 --> 00:00:08,990\n[music playi...",2011-11-11,"After the death of his wife and child, an auth...",15.725,183,4.6,en,"Horror, Thriller",https://image.tmdb.org/t/p/original/t6Wd9gY6e1...
3,12 Strong,"﻿1\n00:01:00,894 --> 00:01:02,942\n<i>WOMAN: F...",2018-01-16,A team of special forces head into Afghanistan...,36.611,2271,6.1,en,"War, Drama, Action, History",https://image.tmdb.org/t/p/original/j18021qCeR...
4,12 Angry Men,"1\n00:01:14,452 --> 00:01:16,745\nThank you ve...",1957-04-10,The defense and the prosecution have rested an...,34.638,6227,8.5,en,Drama,https://image.tmdb.org/t/p/original/e02s4wmTAE...
...,...,...,...,...,...,...,...,...,...,...
3453,You're Next,"﻿1\n00:00:39,806 --> 00:00:41,808\n<font color...",2011-09-10,When the Davison family comes under attack dur...,28.071,1906,6.6,en,"Horror, Thriller, Mystery",https://image.tmdb.org/t/p/original/fuVVOSyXW4...
3454,Zambezia,"﻿1\n00:01:06,000 --> 00:01:07,446\nMarabou's a...",2012-06-05,Set in a bustling bird city on the edge of the...,22.450,223,6.0,en,"Comedy, Animation, Adventure, Family",https://image.tmdb.org/t/p/original/7aJl440Tlb...
3455,Zathura: A Space Adventure,"1\n00:00:05,000 --> 00:00:15,000\nCreated and ...",2005-11-06,"After their father is called into work, two yo...",41.230,2610,6.4,en,"Family, Fantasy, Science Fiction, Adventure, C...",https://image.tmdb.org/t/p/original/g0HLEZfqJp...
3456,Zombeavers,"﻿1\n00:01:02,504 --> 00:01:05,657\nSo four mon...",2014-10-20,College friends find their weekend of sex and ...,22.543,532,4.9,en,"Horror, Action, Comedy",https://image.tmdb.org/t/p/original/gdb7v5GCfi...
